In [1]:
import pandas as pd
import warnings

#warnings.filterwarnings("ignore")

from epilepsy_prediction import load_prediction_model, data_preprocess,fuse_string,evaluate_model,load_imputation_model, evaluate_single_model
from epilepsy_prediction.imputation import column_imputer
from copy import deepcopy
from pandas import read_csv
from pandas import DataFrame
from pandas import melt
from pandas import merge
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression

def load_data(change_name_file,data_file_name,outcome_col="outcome"):
    data_df = pd.read_csv(data_file_name, index_col=0)
    new_name_df = pd.read_csv(change_name_file,index_col=0)
    data_df.rename(new_name_df['marketscan'].to_dict(),inplace=True,axis=1)
    y_train = data_df.loc[data_df["test_train"]=="train",outcome_col]
    X_train = data_df.loc[data_df["test_train"]=="train",~( data_df.columns.isin([outcome_col,"test_train"]))]
    y_test = data_df.loc[~(data_df["test_train"]=="train"),outcome_col]
    X_test = data_df.loc[~(data_df["test_train"]=="train"),~ (data_df.columns.isin([outcome_col,"test_train"]))]
    return X_train,y_train,X_test,y_test

variables = "carba	valpo	topi	lamo	oxca	leve".split()

def get_one_hot_column(Xs, y_test, col_name):
    test_set = Xs[Xs[col_name]==1]
    y_set = y_test[test_set.index]
    return test_set, y_set


## all drugs, outcome any

In [2]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_drugs_all.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_any")

cols=X_train.columns[-6:]
X = X_train.drop("outcome_primary", axis = 1)  
Xs = X_test.drop("outcome_primary", axis = 1)  
X.sort_index(inplace=True)
Xs.sort_index(inplace=True)

stbl = X_train[cols].sum().reset_index().set_axis(['drug', 'count'], axis=1).assign(weight=lambda df: df['count'])

wtbl = merge(
    melt(X_train[cols].reset_index(), id_vars=['id'], var_name='drug')
    .pipe(lambda df: df[df['value'] == 1])
    [['id', 'drug']],
    stbl,
    on='drug',
    how='left'
)[['id', 'weight']].set_index('id')

weights = wtbl.loc[X.index.values, 'weight']

##XGBoost
xgbc = XGBClassifier()
model_xg = xgbc.fit(X, y_train.loc[X.index], sample_weight=weights)

##Logistic
##lr = LogisticRegression()
##model_lr = lr.fit(X, y_train.loc[X.index], sample_weight=weights)

In [3]:
#AUC test
probs_xg = model_xg.predict_proba(Xs)[:, 1]
auc_xg = roc_auc_score(y_test.loc[Xs.index], probs_xg)

evaluate_single_model(model_xg,Xs,y_test.loc[Xs.index],metrics=None)

{'metrics': {'auc1': {'org': 0.5421315404074025, 'mean': 0.5423063533484223, 'std': 0.026633355763493318, 'conf_interval': 95, 'conf_lower': 0.49093329782984957, 'conf_upper': 0.5939555318865664}}}

In [4]:
#AUC factorization
carba_one, carba_test = get_one_hot_column(Xs, y_test, "carba")
valpo_one, valpo_test = get_one_hot_column(Xs, y_test, "valpo")
lamo_one, lamo_test = get_one_hot_column(Xs, y_test, "lamo")
leve_one, leve_test = get_one_hot_column(Xs, y_test, "leve")
topi_one, topi_test = get_one_hot_column(Xs, y_test, "topi")
oxca_one, oxca_test = get_one_hot_column(Xs, y_test, "oxca")


auc_carba = evaluate_single_model(model_xg,carba_one,carba_test,metrics=None)
auc_valpo = evaluate_single_model(model_xg, valpo_one, valpo_test,metrics=None)
auc_lamo = evaluate_single_model(model_xg,lamo_one,lamo_test,metrics=None)
auc_leve = evaluate_single_model(model_xg,leve_one,leve_test,metrics=None)
auc_topi = evaluate_single_model(model_xg,topi_one,topi_test,metrics=None)
auc_oxca = evaluate_single_model(model_xg, oxca_one, oxca_test,metrics=None)

print(auc_carba)
print(auc_valpo)
print(auc_lamo)
print(auc_leve)
print(auc_topi)
print(auc_oxca)

{'metrics': {'auc1': {'org': 0.5461609620721554, 'mean': 0.5456537156953438, 'std': 0.047562741306681304, 'conf_interval': 95, 'conf_lower': 0.45235892691951896, 'conf_upper': 0.6379278445883442}}}
{'metrics': {'auc1': {'org': 0.5538061158100195, 'mean': 0.5540187247885491, 'std': 0.05941342909502394, 'conf_interval': 95, 'conf_lower': 0.4360442420299285, 'conf_upper': 0.6687052700065063}}}
{'metrics': {'auc1': {'org': 0.5055495882563552, 'mean': 0.5055664876476906, 'std': 0.06276063874259119, 'conf_interval': 95, 'conf_lower': 0.38238453276047263, 'conf_upper': 0.6276405298961689}}}
{'metrics': {'auc1': {'org': 0.6340909090909091, 'mean': 0.6335813636363636, 'std': 0.08743719512142689, 'conf_interval': 95, 'conf_lower': 0.4545454545454546, 'conf_upper': 0.7965909090909091}}}
{'metrics': {'auc1': {'org': 0.7843137254901961, 'mean': 0.7856588235294119, 'std': 0.14291206304017542, 'conf_interval': 95, 'conf_lower': 0.45882352941176474, 'conf_upper': 0.9843137254901961}}}
{'metrics': {'au

## all drugs, outcome primary

In [5]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_drugs_all.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_primary")

cols=X_train.columns[-6:]
X = X_train.drop("outcome_any", axis = 1)  
Xs = X_test.drop("outcome_any", axis = 1)  
X.sort_index(inplace=True)
Xs.sort_index(inplace=True)

stbl = X_train[cols].sum().reset_index().set_axis(['drug', 'count'], axis=1).assign(weight=lambda df: df['count'])

wtbl = merge(
    melt(X_train[cols].reset_index(), id_vars=['id'], var_name='drug')
    .pipe(lambda df: df[df['value'] == 1])
    [['id', 'drug']],
    stbl,
    on='drug',
    how='left'
)[['id', 'weight']].set_index('id')

weights = wtbl.loc[X.index.values, 'weight']

##XGBoost
xgbc = XGBClassifier()
model_xg = xgbc.fit(X, y_train.loc[X.index], sample_weight=weights)

##Logistic
##lr = LogisticRegression()
##model_lr = lr.fit(X, y_train.loc[X.index], sample_weight=weights)

In [6]:
#AUC test
probs_xg = model_xg.predict_proba(Xs)[:, 1]
auc_xg = roc_auc_score(y_test.loc[Xs.index], probs_xg)

evaluate_single_model(model_xg,Xs,y_test.loc[Xs.index],metrics=None)

{'metrics': {'auc1': {'org': 0.5830064402810304, 'mean': 0.5832153776346605, 'std': 0.03902160727373693, 'conf_interval': 95, 'conf_lower': 0.5072745901639344, 'conf_upper': 0.6594701405152225}}}

In [7]:
#AUC factorization
carba_one, carba_test = get_one_hot_column(Xs, y_test, "carba")
valpo_one, valpo_test = get_one_hot_column(Xs, y_test, "valpo")
lamo_one, lamo_test = get_one_hot_column(Xs, y_test, "lamo")
leve_one, leve_test = get_one_hot_column(Xs, y_test, "leve")
topi_one, topi_test = get_one_hot_column(Xs, y_test, "topi")
oxca_one, oxca_test = get_one_hot_column(Xs, y_test, "oxca")

auc_carba = evaluate_single_model(model_xg,carba_one,carba_test,metrics=None)
auc_valpo = evaluate_single_model(model_xg, valpo_one, valpo_test,metrics=None)
auc_lamo = evaluate_single_model(model_xg,lamo_one,lamo_test,metrics=None)
auc_leve = evaluate_single_model(model_xg,leve_one,leve_test,metrics=None)
auc_topi = evaluate_single_model(model_xg,topi_one,topi_test,metrics=None)
auc_oxca = evaluate_single_model(model_xg, oxca_one, oxca_test,metrics=None)


print(auc_carba)
print(auc_valpo)
print(auc_lamo)
print(auc_leve)
print(auc_topi)
print(auc_oxca)

{'metrics': {'auc1': {'org': 0.6147876393396359, 'mean': 0.6139995625793706, 'std': 0.07366058058107193, 'conf_interval': 95, 'conf_lower': 0.4666290390856498, 'conf_upper': 0.7543389304360096}}}
{'metrics': {'auc1': {'org': 0.6323979591836735, 'mean': 0.6321942091836734, 'std': 0.06751173279603521, 'conf_interval': 95, 'conf_lower': 0.4948979591836734, 'conf_upper': 0.7579081632653061}}}
{'metrics': {'auc1': {'org': 0.6498084291187739, 'mean': 0.6497557088122606, 'std': 0.08676166041235352, 'conf_interval': 95, 'conf_lower': 0.4766283524904214, 'conf_upper': 0.8122605363984674}}}
{'metrics': {'auc1': {'org': 0.6333333333333333, 'mean': 0.6315166666666667, 'std': 0.12464613759370273, 'conf_interval': 95, 'conf_lower': 0.3692307692307692, 'conf_upper': 0.8461538461538461}}}
{'metrics': {'auc1': {'org': 0.1454545454545455, 'mean': 0.14672363636363636, 'std': 0.04757847961330887, 'conf_interval': 95, 'conf_lower': 0.054545454545454564, 'conf_upper': 0.2545454545454545}}}
{'metrics': {'auc

## 1st index any

In [8]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_drugs_1st_index.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_any")

cols=X_train.columns[-6:]
X = X_train.drop("outcome_primary", axis = 1)  
Xs = X_test.drop("outcome_primary", axis = 1)  
X.sort_index(inplace=True)
Xs.sort_index(inplace=True)

stbl = X_train[cols].sum().reset_index().set_axis(['drug', 'count'], axis=1).assign(weight=lambda df: df['count'])

wtbl = merge(
    melt(X_train[cols].reset_index(), id_vars=['id'], var_name='drug')
    .pipe(lambda df: df[df['value'] == 1])
    [['id', 'drug']],
    stbl,
    on='drug',
    how='left'
)[['id', 'weight']].set_index('id')

weights = wtbl.loc[X.index.values, 'weight']

##XGBoost
xgbc = XGBClassifier()
model_xg = xgbc.fit(X, y_train.loc[X.index], sample_weight=weights)

##Logistic
##lr = LogisticRegression()
##model_lr = lr.fit(X, y_train.loc[X.index], sample_weight=weights)

In [9]:
#AUC test
probs_xg = model_xg.predict_proba(Xs)[:, 1]
auc_xg = roc_auc_score(y_test.loc[Xs.index], probs_xg)

evaluate_single_model(model_xg,Xs,y_test.loc[Xs.index],metrics=None)

{'metrics': {'auc1': {'org': 0.5206204718558425, 'mean': 0.5206520434701839, 'std': 0.03100063917326956, 'conf_interval': 95, 'conf_lower': 0.4603148801783392, 'conf_upper': 0.5814833735835035}}}

In [10]:
#AUC factorization
carba_one, carba_test = get_one_hot_column(Xs, y_test, "carba")
valpo_one, valpo_test = get_one_hot_column(Xs, y_test, "valpo")
lamo_one, lamo_test = get_one_hot_column(Xs, y_test, "lamo")
leve_one, leve_test = get_one_hot_column(Xs, y_test, "leve")
topi_one, topi_test = get_one_hot_column(Xs, y_test, "topi")
oxca_one, oxca_test = get_one_hot_column(Xs, y_test, "oxca")

auc_carba = evaluate_single_model(model_xg,carba_one,carba_test,metrics=None)
auc_valpo = evaluate_single_model(model_xg, valpo_one, valpo_test,metrics=None)
auc_lamo = evaluate_single_model(model_xg,lamo_one,lamo_test,metrics=None)
auc_leve = evaluate_single_model(model_xg,leve_one,leve_test,metrics=None)
auc_topi = evaluate_single_model(model_xg,topi_one,topi_test,metrics=None)
auc_oxca = evaluate_single_model(model_xg, oxca_one, oxca_test,metrics=None)


print(auc_carba)
print(auc_valpo)
print(auc_lamo)
print(auc_leve)
print(auc_topi)
print(auc_oxca)

{'metrics': {'auc1': {'org': 0.5603525211958947, 'mean': 0.5605366577420794, 'std': 0.054482011787623685, 'conf_interval': 95, 'conf_lower': 0.4525881302989737, 'conf_upper': 0.6649933065595717}}}
{'metrics': {'auc1': {'org': 0.5263480392156863, 'mean': 0.5273661458333333, 'std': 0.0776636925408339, 'conf_interval': 95, 'conf_lower': 0.3740808823529411, 'conf_upper': 0.6773897058823529}}}
{'metrics': {'auc1': {'org': 0.521487204249155, 'mean': 0.521469097054563, 'std': 0.08126188526231228, 'conf_interval': 95, 'conf_lower': 0.3626267503621438, 'conf_upper': 0.6788990825688073}}}
{'metrics': {'auc1': {'org': 0.4624113475177305, 'mean': 0.4631248226950354, 'std': 0.08295372346924884, 'conf_interval': 95, 'conf_lower': 0.300709219858156, 'conf_upper': 0.6269503546099291}}}
{'metrics': {'auc1': {'org': 0.5694444444444444, 'mean': 0.5672784722222223, 'std': 0.19456586218927516, 'conf_interval': 95, 'conf_lower': 0.17361111111111116, 'conf_upper': 0.9444444444444445}}}
{'metrics': {'auc1': {

## 1st primary

In [11]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary_19.01.23.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_drugs_1st_index.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name, outcome_col="outcome_primary")

cols=X_train.columns[-6:]
X = X_train.drop("outcome_any", axis = 1)  
Xs = X_test.drop("outcome_any", axis = 1)  
X.sort_index(inplace=True)
Xs.sort_index(inplace=True)

stbl = X_train[cols].sum().reset_index().set_axis(['drug', 'count'], axis=1).assign(weight=lambda df: df['count'])

wtbl = merge(
    melt(X_train[cols].reset_index(), id_vars=['id'], var_name='drug')
    .pipe(lambda df: df[df['value'] == 1])
    [['id', 'drug']],
    stbl,
    on='drug',
    how='left'
)[['id', 'weight']].set_index('id')

weights = wtbl.loc[X.index.values, 'weight']

##XGBoost
xgbc = XGBClassifier()
model_xg = xgbc.fit(X, y_train.loc[X.index], sample_weight=weights)

##Logistic
##lr = LogisticRegression()
##model_lr = lr.fit(X, y_train.loc[X.index], sample_weight=weights)

In [12]:
#AUC test
probs_xg = model_xg.predict_proba(Xs)[:, 1]
auc_xg = roc_auc_score(y_test.loc[Xs.index], probs_xg)

evaluate_single_model(model_xg,Xs,y_test.loc[Xs.index],metrics=None)

{'metrics': {'auc1': {'org': 0.5849262317899229, 'mean': 0.5841299039621416, 'std': 0.04426138486201783, 'conf_interval': 95, 'conf_lower': 0.4962651943954718, 'conf_upper': 0.6695740929757817}}}

In [13]:
#AUC factorization
carba_one, carba_test = get_one_hot_column(Xs, y_test, "carba")
valpo_one, valpo_test = get_one_hot_column(Xs, y_test, "valpo")
lamo_one, lamo_test = get_one_hot_column(Xs, y_test, "lamo")
leve_one, leve_test = get_one_hot_column(Xs, y_test, "leve")
topi_one, topi_test = get_one_hot_column(Xs, y_test, "topi")
oxca_one, oxca_test = get_one_hot_column(Xs, y_test, "oxca")

auc_carba = evaluate_single_model(model_xg,carba_one,carba_test,metrics=None)
auc_valpo = evaluate_single_model(model_xg, valpo_one, valpo_test,metrics=None)
auc_lamo = evaluate_single_model(model_xg,lamo_one,lamo_test,metrics=None)
auc_leve = evaluate_single_model(model_xg,leve_one,leve_test,metrics=None)
auc_topi = evaluate_single_model(model_xg,topi_one,topi_test,metrics=None)
auc_oxca = evaluate_single_model(model_xg, oxca_one, oxca_test,metrics=None)


print(auc_carba)
print(auc_valpo)
print(auc_lamo)
print(auc_leve)
print(auc_topi)
print(auc_oxca)

{'metrics': {'auc1': {'org': 0.5443773234200744, 'mean': 0.5449388475836431, 'std': 0.08316841174616477, 'conf_interval': 95, 'conf_lower': 0.375, 'conf_upper': 0.7039962825278812}}}
{'metrics': {'auc1': {'org': 0.6432160804020101, 'mean': 0.6415017085427136, 'std': 0.09907551169492221, 'conf_interval': 95, 'conf_lower': 0.4381909547738693, 'conf_upper': 0.8261306532663316}}}
{'metrics': {'auc1': {'org': 0.7394957983193278, 'mean': 0.7393047619047618, 'std': 0.07815766756107041, 'conf_interval': 95, 'conf_lower': 0.5714285714285714, 'conf_upper': 0.8786181139122317}}}
{'metrics': {'auc1': {'org': 0.5416666666666667, 'mean': 0.5415928571428572, 'std': 0.14193328455559404, 'conf_interval': 95, 'conf_lower': 0.26785714285714285, 'conf_upper': 0.8125}}}
{'metrics': {'auc1': {'org': 0.717948717948718, 'mean': 0.7168999999999999, 'std': 0.07187736647906497, 'conf_interval': 95, 'conf_lower': 0.5641025641025641, 'conf_upper': 0.8461538461538461}}}
{'metrics': {'auc1': {'org': 0.50272108843537